In [ ]:
from pathlib import Path

import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import (
    f1_score,
    roc_auc_score,
    roc_curve,
    auc,
    precision_recall_curve,
    precision_recall_fscore_support,
    confusion_matrix
)
from catboost import Pool
import mlflow
from mlflow.types.schema import Schema, ColSpec
import optuna
import shap
shap.initjs()
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt

from var import DATA_OUT, IMAGE_OUT, FORECAST_HOURS_IN_ADVANCE
from src.backend import ML_SERVER_URI, EXPERIMENT_NAME
from src.model.opt import objective
from src.model.model import (
    get_or_create_experiment,
    start_crossvalidated_run,
    evaluate_crossvalidated_metrics,
)

# logging level: WARNING (INFO logs are suppressed)
optuna.logging.set_verbosity(optuna.logging.WARNING)

# MLFlow web server URI
mlflow.set_tracking_uri(ML_SERVER_URI)

In [ ]:
df = pd.read_pickle(Path(DATA_OUT, 'df_dataset.pickle'))

In [ ]:
X = df[
    [
        'ie_fix',
        'ie_variation',
        'ie_mav_3h',
        'ie_mav_12h',
        'iu_fix',
        'iu_variation',
        'iu_mav_3h',
        'iu_mav_12h',
        'hf',
        'hf_mav_2h',
        'f_107_adj',
        'hp_30',
        'dst',
        'solar_zenith_angle',
        'newell',
        'bz',
        'speed',
        'rho',
        *[col_ for col_ in df.columns if col_.startswith('spectral_contribution_')],
        *[col_ for col_ in df.columns if col_.startswith('azimuth_')],
        *[col_ for col_ in df.columns if col_.startswith('velocity_')],
    ]
].copy()

target = f'tid_within_{FORECAST_HOURS_IN_ADVANCE}h'

y = df[target].copy()

In [ ]:
cat_features = [
    'ie_variation',
    'iu_variation',
]

static_params = {
    "eval_metric": 'F1',
    "random_seed": 42,
    "cat_features": cat_features,
    "od_type": "Iter",
    "use_best_model": True,
    "has_time": True,
    "od_wait": 200,
}

In [ ]:
# n_days_for_testing = 365
# ts_cv = TimeSeriesSplit(n_splits=5, test_size=n_days_for_testing*24*2)

ts_cv = TimeSeriesSplit(n_splits=5)

In [ ]:
# corr = df.select_dtypes(exclude=['object'])[
#     [
#         'ie_fix',
#         'iu_fix',
#         'hf',
#         'hp_30',
#         'dst',
#         'newell',
#         'bz',
#         'speed',
#         'rho',
#         'solar_zenith_angle',
#         'tid_within_3h',
#     ]
# ].corr()

# corr.style.background_gradient(cmap='coolwarm')

## Optuna (hyper-params optimisation)

In [ ]:
obj = lambda trial: objective(
    trial,
    X=X,
    y=y,
    cv=ts_cv,
    params=static_params,
)

study = optuna.create_study(study_name='catboost_clf', direction='maximize')
study.optimize(obj, n_trials=10, show_progress_bar=True)

trial = study.best_trial

In [ ]:
print("Best trial:")
print(f"  F1: {trial.value:.3f}")

print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

In [ ]:
# optuna.visualization.plot_optimization_history(study)

In [ ]:
# optuna.visualization.plot_param_importances(study)

In [ ]:
# Fine-tuned hyperparams
best_params = study.best_params
static_params.update(best_params)

## Fine-tuned and/or cross-validated model

In [ ]:
mlflow.set_experiment(
    experiment_id=get_or_create_experiment(EXPERIMENT_NAME)
);

### Pre-trained model (retrieved by `run_id`)

In [ ]:
cat_model, (f1s, prs, rcs) = start_crossvalidated_run(
    X=X,
    y=y,
    time_series_cross_validator=ts_cv,
    run_id='47379e58abff4b18989898a5b6ecbe08',
)

### Train a model from scratch, usually after h-params optimisation

**[Model signatures](https://www.mlflow.org/docs/latest/models.html#model-signature)** define what the model expects (input, output and parameters) and enforce it later in deployment.

Signatures are fetched by the Tracking UI and Model Registry UI to display model inputs, outputs and params; they are also utilized by MLflow model deployment tools to validate inference inputs according to the model’s assigned signature

In [ ]:
input_schema = Schema(
    [ColSpec("double", col_) for col_ in X.columns]
)

output_schema = Schema(
    [ColSpec("long", y.name)]
)

signature = mlflow.models.ModelSignature(inputs=input_schema, outputs=output_schema)

cat_model, (f1s, prs, rcs) = start_crossvalidated_run(
    X=X,
    y=y,
    time_series_cross_validator=ts_cv,
    model_params=static_params,
    model_signature=signature,
)

### Train a model *without* MLFlow (not suggested)

In [ ]:
from src.model.model import instantiate_and_fit_model

f1s, prs, rcs = [], [], []
for i, (train_idx, test_idx) in enumerate(
        ts_cv.split(X)
    ):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        cat_model = instantiate_and_fit_model(
            X_train=X_train,
            y_train=y_train,
            X_test=X_test,
            y_test=y_test,
            loss_function="Logloss",
            params=static_params,
        )

        y_pred = cat_model.predict(X_test)
        # Evaluate in-fold metrics
        p, r, f, _ = precision_recall_fscore_support(y_test, y_pred)
        f1s.append(f[1])
        prs.append(p[1])
        rcs.append(r[1])

In [ ]:
cat_model.save_model('../src/assets/model.cb')

### Evaluate classification metrics

In [ ]:
weights = [tr_idx.shape[0] / X.shape[0] for tr_idx, _ in ts_cv.split(X)]

evaluate_crossvalidated_metrics(
    metrics={
        'F1-score': f1s,
        'Precision': prs,
        'Recall': rcs,
    },
    weights=weights,
)

In [ ]:
train_idx, test_idx = [(tr_idx, ts_idx) for (tr_idx, ts_idx) in ts_cv.split(X)][-1]

X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

y_pred = cat_model.predict(X_test)

## SHAP

In [ ]:
shap_values = cat_model.get_feature_importance(
    Pool(X_test, label=y_test, cat_features=cat_features),
    type="ShapValues",
)

shap_values = shap_values[:,:-1]

In [ ]:
# df_shap = pd.DataFrame(
#     (
#         zip(
#             X_train.columns[np.argsort(np.abs(shap_values).mean(0))][::-1],
#             -np.sort(-np.abs(shap_values).mean(0))
#         )
#     ),
#     columns=["feature", "importance"],
# )

# df_shap.to_pickle(Path(DATA_OUT, 'df_feat_imp.pickle'))

In [ ]:
VAR_NAMES_DICT = {
    "ie_fix": "IE",
    "ie_variation": "IE - variation",
    "ie_mav_3h": "IE - 3h MAv",
    "ie_mav_12h": "IE - 12h MAv",
    "iu_fix": "IU",
    "iu_variation": "IU - variation",
    "iu_mav_3h": "IU - 3h MAv",
    "iu_mav_12h": "IU - 12h MAv",
    "hf": "HF-EU",
    "hf_mav_2h": "HF-EU - 2h MAv",
    "f_107_adj": "F10.7",
    "hp_30": "HP-30",
    "dst": "Dst",
    "solar_zenith_angle": "Solar zenith angle",
    "newell": "Newell coupling",
    "rho": "ρ (solar wind dens.)",
    "vx": "vx (solar wind vel.)",
    "bz": "Bz (IMF)",
    "spectral_contribution_at": "Spectral contribution (Athens)",
    "spectral_contribution_ff": "Spectral contribution (Fairford)",
    "spectral_contribution_jr": "Spectral contribution (Juliusruh)",
    "spectral_contribution_pq": "Spectral contribution (Průhonice)",
    "spectral_contribution_ro": "Spectral contribution (Rome)",
    "spectral_contribution_vt": "Spectral contribution (San Vito)",
    "azimuth_at": "Azimuth (Athens)",
    "azimuth_ff": "Azimuth (Fairford)",
    "azimuth_jr": "Azimuth (Juliusruh)",
    "azimuth_pq": "Azimuth (Průhonice)",
    "azimuth_ro": "Azimuth (Rome)",
    "azimuth_vt": "Azimuth (San Vito)",
    "velocity_at": "Velocity (Athens)",
    "velocity_ff": "Velocity (Fairford)",
    "velocity_jr": "Velocity (Juliusruh)",
    "velocity_pq": "Velocity (Průhonice)",
    "velocity_ro": "Velocity (Rome)",
    "velocity_vt": "Velocity (San Vito)",
#     "h_55_diff": "Bh variation (55°-60°)",
#     "h_60_diff": "Bh variation (60°-65°)",
#     "h_65_diff": "Bh variation (65°-70°)",
#     "h_70_diff": "Bh variation (70°-75°)",
#     "h_75_diff": "Bh variation (75°-80°)",
}

In [ ]:
shap.summary_plot(
    shap_values,
    X_test.rename(columns=VAR_NAMES_DICT),
    max_display=12,
#     plot_type='bar',
    show=False,
)

# plt.savefig(
#     Path(IMAGE_OUT, 'shap_summary.png'),
#     dpi=400,
# )

In [ ]:
shap.dependence_plot(
    'Dst',
    shap_values,
    X_test.rename(columns=VAR_NAMES_DICT),
    interaction_index='HF-EU',
#     show=False,
)

# plt.savefig(
#     Path(IMAGE_OUT, 'shap_dp.png'),
#     dpi=400,
#     bbox_inches='tight'
# )

In [ ]:
explainer = shap.TreeExplainer(cat_model)
shap_values_ = explainer.shap_values(X_test)

In [ ]:
row = X_test.index.get_loc('2022-01-04 16:00')

shap.force_plot(
    explainer.expected_value,
    shap_values[row,:],
    X_test.iloc[row,:],
    link='logit',
)

In [ ]:
# X_test.reset_index().loc[25880:25916].reset_index()

In [ ]:
# Stacked force plot

shap.force_plot(
    explainer.expected_value,
    shap_values[25880:25916],
    X_test.reset_index().loc[25880:25916].set_index('datetime'),
    link='logit',
)

In [ ]:
y_test.iloc[row], y_pred[row]

In [ ]:
shap.plots.bar(
    explainer(X_test),
#    show=False,
    max_display=15,
)

# plt.savefig(
#     Path(IMAGE_OUT, 'shap_bar_plot.png'),
#     dpi=300,
#     bbox_inches='tight',
# )

## Confusion matrix

In [ ]:
import seaborn as sns

conf_matrix = confusion_matrix(y_test, y_pred, normalize='true')

plt.figure(figsize=(8, 8))

sns.set(font_scale=2.4)

sns.heatmap(
    conf_matrix,
    annot=True,
    fmt='.2f',
    cmap='Blues',
    cbar=False,
    xticklabels=['TID not predicted', 'TID predicted'],
    yticklabels=['TID doesn\'t occur', 'TID occurs'],
)

# plt.savefig(
#     Path(IMAGE_OUT, f'confusion_matrix.png', dpi=500, bbox_inches='tight')
# )

plt.show()

## Evaluation of classification

In [ ]:
df_eval = X_test.copy(deep=True)
df_eval['true'] = y_test
df_eval['pred'] = cat_model.predict(X_test)
df_eval['pred_proba'] = cat_model.predict_proba(X_test)[:,1]

### ROC curve

In [ ]:
fpr, tpr, thresholds = roc_curve(
    df_eval['true'],
    df_eval['pred_proba'],
)

In [ ]:
roc_auc = roc_auc_score(
    df_eval['true'],
    df_eval['pred_proba'],
)

In [ ]:
fig = px.scatter(x=fpr, y=tpr)

fig.update_layout(
    height=700,
    width=800,
    autosize=False,
    shapes=[
        dict(
            type='line',
            x0=0,
            y0=0,
            x1=1,
            y1=1,
            line=dict(color='navy', width=2, dash='dash'),
        )
    ],
    title=f'ROC Curve (ROC-AUC: <b>{roc_auc:.2f}</b>)',
    xaxis=dict(title='False Positive Rate'),
    yaxis=dict(title='True Positive Rate'),
    template='ggplot2',
)

# fig.write_html(
#     Path(IMAGE_OUT,f'plot_roc_curve.html')
# )

fig.show()

### PR curve

In [ ]:
p, r, t = precision_recall_curve(
    df_eval['true'],
    df_eval['pred_proba'],
    drop_intermediate=True,
)

In [ ]:
pr_auc = auc(r, p)

In [ ]:
f1_scores = 2 * (p*r) / (p+r)
thr_f1_max = t[np.argmax(f1_scores)]

print(
    f'{np.round(thr_f1_max, 3)} is the threshold that maximises F1-score to {np.round(f1_scores[np.argmax(f1_scores)], 3)}'
)

In [ ]:
np.where(np.logical_and(p>=0.80, p<0.80001))

In [ ]:
idx = 3900
thr_p_80 = t[idx]

print(
    f'{thr_p_80.round(3)} is the threshold that gives a precision of {p[idx].round(3)} (recall: {r[idx].round(3)} | F1-score: {f1_scores[idx].round(3)})'
)

In [ ]:
np.where(np.logical_and(r>=0.60, r<0.6005))

In [ ]:
idx = 2495
thr_r_60 = t[idx]

print(
    f'{thr_r_60.round(3)} is the threshold that gives a recall of {r[idx].round(3)} (precision: {p[idx].round(3)} | F1-score: {f1_scores[idx].round(3)})'
)

In [ ]:
thr_f1_max.round(3), thr_p_80.round(3), thr_r_60.round(3)

In [ ]:
df_eval['pred_f1_max'] = np.where(
    df_eval['pred_proba'].gt(thr_f1_max),
    1,
    0,
)

In [ ]:
df_eval['pred_p_80'] = np.where(
    df_eval['pred_proba'].gt(thr_p_80),
    1,
    0,
)

In [ ]:
df_eval['pred_r_60'] = np.where(
    df_eval['pred_proba'].gt(thr_r_60),
    1,
    0,
)

In [ ]:
for lab_ in ['pred_f1_max', 'pred_p_80', 'pred_r_60']:

    conf_matrix = confusion_matrix(df_eval['true'], df_eval[lab_], normalize='true')

    plt.figure(figsize=(8, 8))
    sns.set(font_scale=2)

    sns.heatmap(
        conf_matrix,
        annot=True,
        fmt='.1%',
        cmap='Blues',
        cbar=False,
        xticklabels=['TID not predicted', 'TID predicted'],
        yticklabels=['TID doesn\'t occur', 'TID occurs'],
        annot_kws={"size": 32},
    )

#     plt.savefig(
#         Path(IMAGE_OUT, f'confusion_matrix_{lab_}.png', dpi=500, bbox_inches='tight')
#     )

    plt.show()

In [ ]:
# df_web_app = df_eval.copy(deep=True)

# float_cols = df_web_app.select_dtypes(include=[np.float64]).columns
# int_cols = df_web_app.select_dtypes(include=[np.int32, np.int64]).columns

# df_web_app[float_cols] = df_web_app[float_cols].astype(np.float32)
# df_web_app[int_cols] = df_web_app[int_cols].astype(np.int8)

# df_web_app.to_pickle(Path(DATA_OUT, 'df_eval.pickle'))

In [ ]:
fig = px.scatter(x=r, y=p)

fig.add_shape(
    type='line',
    x0=0,
    y0=1,
    x1=1,
    y1=0,
    line=dict(color='navy', width=2, dash='dash'),
)

fig.add_shape(
    type='line',
    x0=r[idx],
    y0=0,
    x1=r[idx],
    y1=p[idx],
    line=dict(color='red', width=2, dash='dash'),
)

fig.add_shape(
    type='line',
    x0=0,
    y0=p[idx],
    x1=r[idx],
    y1=p[idx],
    line=dict(color='red', width=2, dash='dash'),
)

DOT_SIZE = 15
fig.add_trace(
    go.Scatter(
        x=[r[idx]],
        y=[p[idx]],
        mode='markers',
        marker=dict(color='red', size=DOT_SIZE),
        showlegend=False,
    )
)

fig.update_layout(
    height=700,
    width=800,
    autosize=False,
    title=f'PR Curve (PR-AUC: <b>{pr_auc:.2f}</b>)',
    xaxis=dict(title='Recall'),
    yaxis=dict(title='Precision'),
    template='ggplot2',
)

# fig.write_html(
#     Path(IMAGE_OUT,f'plot_pr_curve.html')
# )

fig.show()